In [2]:
# LangChain Tools


In [4]:
## What are Tools in LangChain?
"""
A **Tool** is a function you wrap so that the LLM (language model) can call it when needed.

**Think of a tool as a helper function the AI can use.**

### Why Tools?

LLMs are good at language but weak at:
- Math
- Web search
- APIs
- Database lookup

Tools fix this!
"""


'\nA **Tool** is a function you wrap so that the LLM (language model) can call it when needed.\n\n**Think of a tool as a helper function the AI can use.**\n\n### Why Tools?\n\nLLMs are good at language but weak at:\n- Math\n- Web search\n- APIs\n- Database lookup\n\nTools fix this!\n'

In [5]:
# STEP 1: Import Tool class
from langchain.agents import Tool


In [7]:
# STEP 2: Define a basic calculator function
def simple_calculator(query: str) -> str:
    try:
        result = eval(query)  #
        return f"The result is: {result}"
    except Exception as e:
        return f"Error: {str(e)}"


In [8]:
# STEP 3: Wrap it into a LangChain Tool
calculator_tool = Tool(
    name="Calculator",
    func=simple_calculator,
    description="Useful for doing math calculations. Input should be a valid math expression."
)


In [12]:
# STEP 4 & 5: Create the Agent with the Tool and Initialize

In [10]:
!pip install -q langchain langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 2.6 MB/s eta 0:00:00


In [11]:
from langchain.agents import initialize_agent
from langchain.agents.agent_types import AgentType
from langchain_openai import ChatOpenAI
from google.colab import userdata
import os
from langchain.agents import Tool, initialize_agent


# Set your OpenAI API Key
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

# STEP 4: Create the LLM
llm = ChatOpenAI(
    model="gpt-4.1-nano",
    temperature=0
)

# STEP 5: Initialize the agent with the tool
agent = initialize_agent(
    tools=[calculator_tool],            # list of Tool objects
    llm=llm,                             # your LLM
    agent=AgentType.OPENAI_FUNCTIONS,   # type of agent
    verbose=True                        # prints reasoning steps
)


/tmp/ipython-input-3075810662.py:19: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [13]:
# STEP 6: Run the Agent (Tool Calling)

In [14]:
# STEP 6: Ask a question that requires the tool
response = agent.run("What is (12 * 4) + 7?")
print("Agent Response:", response)


/tmp/ipython-input-49192643.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent.run("What is (12 * 4) + 7?")




> Entering new AgentExecutor chain...

Invoking: `Calculator` with `(12 * 4) + 7`


The result is: 55The result of (12 * 4) + 7 is 55.

> Finished chain.
Agent Response: The result of (12 * 4) + 7 is 55.


In [15]:
# End of the Notebook


In [16]:
# Example 2 - Words Meaning Tool

In [23]:
# ✅ STEP 0: Install required packages (if running in Colab)
!pip install -q langchain langchain-openai

# ✅ STEP 1: Imports and OpenAI API Key Setup
from langchain_openai import ChatOpenAI
from langchain.agents import Tool, initialize_agent
from langchain.agents.agent_types import AgentType
import os

# Use this if running in Google Colab with secrets stored
try:
    from google.colab import userdata
    os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
except:
    # If not in Colab, manually set your key below
    os.environ["OPENAI_API_KEY"] = "your-openai-key-here"

# ✅ STEP 2: Define the dictionary function
def get_definition(word: str) -> str:
    dictionary = {
        "ai": "AI stands for Artificial Intelligence. It refers to the simulation of human intelligence in machines.",
        "genai": "GenAI stands for Generative AI. It can generate text, images, or other content.",
        "python": "Python is a high-level, interpreted programming language known for its simplicity.",
        "data": "Data is a collection of facts, such as numbers, words, measurements, or observations.",
        "model": "In AI, a model is a mathematical representation trained to make predictions or decisions."
    }

    word_lower = word.lower()
    return dictionary.get(word_lower, f"Sorry, I don't have a definition for '{word}' right now.")

# ✅ STEP 3: Wrap the function into a LangChain Tool (NO SPACES in name!)
dictionary_tool = Tool(
    name="dictionary_tool",  # ✅ Fixed: no spaces or special chars
    func=get_definition,
    description="Useful for finding simple definitions of tech terms. Input should be a single word like 'AI' or 'Python'."
)

# ✅ STEP 4: Initialize LLM and Agent
llm = ChatOpenAI(model="gpt-4.1-nano", temperature=0)

agent = initialize_agent(
    tools=[dictionary_tool],
    llm=llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True
)

# ✅ STEP 5: Ask a question and get response
query = "What is cricket?"
response = agent.run(query)
print("\n💬 Agent response:", response)




> Entering new AgentExecutor chain...

Invoking: `dictionary_tool` with `cricket`


Sorry, I don't have a definition for 'cricket' right now.Cricket is a popular sport played between two teams, usually of eleven players each. It involves batting, bowling, and fielding, with the main objective of scoring runs. The game is played on a large oval-shaped field with a rectangular 22-yard pitch at the center. It is especially popular in countries like India, England, Australia, and Pakistan.

> Finished chain.

💬 Agent response: Cricket is a popular sport played between two teams, usually of eleven players each. It involves batting, bowling, and fielding, with the main objective of scoring runs. The game is played on a large oval-shaped field with a rectangular 22-yard pitch at the center. It is especially popular in countries like India, England, Australia, and Pakistan.


In [25]:
# Example - Weather Agent with User Input

In [29]:
# STEP 1: Install dependencies
!pip install -q langchain langchain-openai requests

# STEP 2: Imports and API Key
import os
import requests
from google.colab import userdata
from langchain.agents import Tool, initialize_agent
from langchain.agents.agent_types import AgentType
from langchain_openai import ChatOpenAI

# STEP 3: Set your OpenAI API key from userdata
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")  # 🔐 Secure API Key

# STEP 4: Define the Weather API tool function
def get_weather(city: str) -> str:
    try:
        geo_url = f"https://geocoding-api.open-meteo.com/v1/search?name={city}"
        geo_response = requests.get(geo_url).json()
        if "results" not in geo_response:
            return f"City '{city}' not found."

        lat = geo_response["results"][0]["latitude"]
        lon = geo_response["results"][0]["longitude"]

        weather_url = f"https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}&current_weather=true"
        weather_response = requests.get(weather_url).json()
        weather = weather_response["current_weather"]
        return f"The temperature in {city} is {weather['temperature']}°C with wind speed {weather['windspeed']} km/h."
    except Exception as e:
        return f"Failed to fetch weather: {str(e)}"

# STEP 5: Wrap in a LangChain Tool
weather_tool = Tool(
    name="weather_tool",
    func=get_weather,
    description="Fetch current weather for a given city. Input should be a city name."
)

# STEP 6: Initialize the Agent
llm = ChatOpenAI(model="gpt-4.1-nano", temperature=0)
agent = initialize_agent(
    tools=[weather_tool],
    llm=llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True
)

# STEP 7: Get user input
city = input("Enter city name for weather info: ").strip()
response = agent.run(f"What’s the weather in {city}?")
print("\n💬 Agent response:", response)


Enter city name for weather info: Mumbai


> Entering new AgentExecutor chain...

Invoking: `weather_tool` with `Mumbai`


The temperature in Mumbai is 28.2°C with wind speed 8.4 km/h.The current weather in Mumbai is 28.2°C with a wind speed of 8.4 km/h.

> Finished chain.

💬 Agent response: The current weather in Mumbai is 28.2°C with a wind speed of 8.4 km/h.


In [30]:
# Example - Weather Agent with User Input for any city in the world along with current time in IST

In [33]:
# STEP 1: Install dependencies
!pip install -q langchain langchain-openai requests pytz

# STEP 2: Imports and API Key
import os
import requests
from datetime import datetime
import pytz
from google.colab import userdata
from langchain.agents import Tool, initialize_agent
from langchain.agents.agent_types import AgentType
from langchain_openai import ChatOpenAI

# STEP 3: Set OpenAI API Key securely
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")  # 🔐

# STEP 4: Define the main tool function
def get_weather_and_time(city: str) -> str:
    try:
        # Geocoding API to get lat/lon
        geo_url = f"https://geocoding-api.open-meteo.com/v1/search?name={city}"
        geo_response = requests.get(geo_url).json()

        if "results" not in geo_response:
            return f"City '{city}' not found."

        lat = geo_response["results"][0]["latitude"]
        lon = geo_response["results"][0]["longitude"]
        name = geo_response["results"][0]["name"]
        country = geo_response["results"][0].get("country", "")

        # Weather API
        weather_url = f"https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}&current_weather=true"
        weather_response = requests.get(weather_url).json()
        weather = weather_response["current_weather"]
        temp = weather["temperature"]
        wind = weather["windspeed"]

        # IST time
        ist_time = datetime.now(pytz.timezone("Asia/Kolkata")).strftime("%I:%M %p")

        return (f"📍 {name}, {country}\n"
                f"🌡️ Temperature: {temp}°C\n"
                f"💨 Wind Speed: {wind} km/h\n"
                f"🕒 Current IST Time: {ist_time}")
    except Exception as e:
        return f"Failed to fetch info: {str(e)}"

# STEP 5: Wrap the function in a LangChain Tool
weather_time_tool = Tool(
    name="weather_time_tool",
    func=get_weather_and_time,
    description="Provides current weather and IST time for a given city. Input should be a city name."
)

# STEP 6: Initialize LangChain Agent
llm = ChatOpenAI(model="gpt-4.1-nano", temperature=0)
agent = initialize_agent(
    tools=[weather_time_tool],
    llm=llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True
)

# STEP 7: Take user input and invoke agent
city = input("🌍 Enter a city name: ").strip()
response = agent.run(f"What's the weather and IST time in {city}?")
print("\n💬 Agent response:\n", response)


🌍 Enter a city name: London


> Entering new AgentExecutor chain...

Invoking: `weather_time_tool` with `London`


📍 London, United Kingdom
🌡️ Temperature: 20.0°C
💨 Wind Speed: 8.2 km/h
🕒 Current IST Time: 04:50 PMThe current weather in London is 20°C with a wind speed of 8.2 km/h. The IST time in London is 04:50 PM.

> Finished chain.

💬 Agent response:
 The current weather in London is 20°C with a wind speed of 8.2 km/h. The IST time in London is 04:50 PM.


In [34]:
# More Dynamic Example

In [38]:
# STEP 1: Install dependencies
!pip install -q langchain langchain-openai requests pytz

# STEP 2: Imports and API Key
import os
import requests
from datetime import datetime
import pytz
from google.colab import userdata
from langchain.agents import Tool, initialize_agent
from langchain.agents.agent_types import AgentType
from langchain_openai import ChatOpenAI

# STEP 3: Set your OpenAI API Key
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

# STEP 4: Define the tool
def get_weather_and_times(city: str) -> str:
    try:
        # Geocode to get latitude, longitude, timezone
        geo_url = f"https://geocoding-api.open-meteo.com/v1/search?name={city}"
        geo_response = requests.get(geo_url).json()
        if "results" not in geo_response:
            return f"❌ City '{city}' not found."

        info = geo_response["results"][0]
        name = info["name"]
        country = info.get("country", "")
        lat = info["latitude"]
        lon = info["longitude"]
        timezone = info.get("timezone", "UTC")

        # Get weather data
        weather_url = f"https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}&current_weather=true"
        weather_data = requests.get(weather_url).json()
        current = weather_data["current_weather"]
        temp = current["temperature"]
        wind = current["windspeed"]

        # Local time at city
        city_time = datetime.now(pytz.timezone(timezone)).strftime("%I:%M %p")

        # Current IST time
        ist_time = datetime.now(pytz.timezone("Asia/Kolkata")).strftime("%I:%M %p")

        return (f"📍 {name}, {country}\n"
                f"🕑 Local Time: {city_time} ({timezone})\n"
                f"🕒 IST Time: {ist_time}\n"
                f"🌡️ Temperature: {temp}°C\n"
                f"💨 Wind: {wind} km/h")
    except Exception as e:
        return f"⚠️ Error: {str(e)}"

# STEP 5: Wrap tool
weather_tool = Tool(
    name="weather_time_tool",
    func=get_weather_and_times,
    description="Get weather, local time, and IST time. Input should be a city name."
)

# STEP 6: Create Agent
llm = ChatOpenAI(model="gpt-4.1-nano", temperature=0)
agent = initialize_agent(
    tools=[weather_tool],
    llm=llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True
)

# STEP 7: Run in loop
print("🌍 Ask for any city's weather & time. Type 'exit' to quit.\n")
while True:
    city = input("Enter city name: ").strip()
    if city.lower() in ["exit", "quit"]:
        print("👋 Exiting. Stay informed!")
        break
    response = agent.run(f"Get weather and time in {city}")
    print("\n💬 Agent response:\n", response, "\n" + "-"*60 + "\n")


🌍 Ask for any city's weather & time. Type 'exit' to quit.

Enter city name: London


> Entering new AgentExecutor chain...

Invoking: `weather_time_tool` with `London`


📍 London, United Kingdom
🕑 Local Time: 12:28 PM (Europe/London)
🕒 IST Time: 04:58 PM
🌡️ Temperature: 20.0°C
💨 Wind: 8.2 km/hThe current weather in London is 20.0°C with a wind speed of 8.2 km/h. The local time in London is 12:28 PM, and the IST time is 04:58 PM.

> Finished chain.

💬 Agent response:
 The current weather in London is 20.0°C with a wind speed of 8.2 km/h. The local time in London is 12:28 PM, and the IST time is 04:58 PM. 
------------------------------------------------------------

Enter city name: New York


> Entering new AgentExecutor chain...

Invoking: `weather_time_tool` with `New York`


📍 New York, United States
🕑 Local Time: 07:28 AM (America/New_York)
🕒 IST Time: 04:58 PM
🌡️ Temperature: 21.6°C
💨 Wind: 9.9 km/hThe current local time in New York is 07:28 AM, and the temperature is 21.6°C with 

In [39]:
# End of the Notebook